# C.V. Agents

## Optimizing a Candidate C.V. for a Job Posting

1. Analyze the job posting to extract structured requirements.
2. Profile a candidate by querying a knowledge base for data relevant to the job posting
3. Optimize the candidate's CV based on the job requirements and profile data

## The Data Flow

1. **Job posting URL → Job Analyst → Structured job requirements:**  
   The job posting URL is analyzed to extract a structured list of requirements.
2. **Candidate data + Job requirements → Candidate Profiler → Profile analysis:**  
   Candidate data is assessed in the context of the job requirements to generate a candidate profile.
3. **Profile + Job requirements → CV Strategist → Optimization recommendations:**  
   The candidate profile and job requirements are used to produce recommendations for CV optimization.
4. **Optimized data → Template engine → LaTeX → PDF:**  
   The optimized CV data can be rendered into a LaTeX template to generate a PDF.

## CrewAI Source Code and YAML Configuration

In [ ]:
from IPython.display import display, Markdown

def display_code(file_path, lang='python'):
    with open(file_path) as file:
        content = file.read()
    display(Markdown(f'#### `{file_path}`\n\n```{lang}\n' + content + '\n```'))


The following cells display the YAML and Python code used to set up the CrewAI agents and tasks for this workflow.

### Agents and Tasks

Agents and tasks are defined using a hybrid YAML and Python approach.

YAML is used for string based configuration

In [ ]:
display_code('optimizer/config/agents.yaml', 'yaml')
display_code('optimizer/config/tasks.yaml', 'yaml')

Python handles structured outputs and tools

In [ ]:
display_code('optimizer/agents.py')
display_code('optimizer/tasks.py')

### Structured Outputs

Pydantic models specify structured outputs.

In [ ]:
display_code('optimizer/models.py')

### The Crew

The `CrewBase` class coordinates the agents and tasks, using decorators to designate special methods for the workflow.

In [ ]:
display_code('optimizer/crew.py')

## Running the Crew

In [ ]:
from optimizer.crew import CvOptimizer

job_posting_url = "https://app.welcometothejungle.com/dashboard/jobs/oA1SArxV"
cv_data_path = "data/cv.json"
output_directory = "job_postings/automattic"

inputs = {
    "job_posting_url": job_posting_url,
    "cv_data_path": cv_data_path,
    "output_directory": output_directory
}

CvOptimizer().crew().kickoff(inputs=inputs)